In [1]:
%pip install -qU langchain langgraph langchain-openai langchain-community langchain-text-splitters chromadb pypdf


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
OPENAI_API_KEY = "<OPENAI_KEY>"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
from claim_verifier import build_retriever, make_search_tool, SYSTEM_PROMPT

from langchain_openai import ChatOpenAI

In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve()))

from claim_verifier import build_retriever, make_search_tool, SYSTEM_PROMPT

PDF_PATH = "data/attention_all_u_need.pdf"

retriever = build_retriever(
    pdf_path=PDF_PATH,
    persist_dir="chroma_claim_db",
    collection_name="claim_pdf",
    k=5,
)
claim_tool = make_search_tool(retriever)
print("Tool name:", claim_tool.name)


/Users/satyammittal/github/AI-ML/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/satyammittal/github/AI-ML/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/satyammittal/github/AI-ML/demo_applications/agent-patterns-pack/claim_verifier.py:57: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vs = Chroma(


Tool name: claim_verifier


In [3]:
def call_claim_tool(query: str) -> str:
    # Try common invoke patterns across versions
    try:
        return claim_tool.invoke({"query": query})
    except Exception:
        try:
            return claim_tool.invoke(query)
        except Exception:
            # Fallback: call underlying function directly (rare)
            return claim_tool.func(query)


In [5]:
draft_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def draft_answer(question: str) -> dict:
    evidence = call_claim_tool(question)
    prompt = f"""
{SYSTEM_PROMPT}

Tool name: {claim_tool.name}

Question:
{question}

Evidence (cited snippets):
{evidence}

Write a DRAFT answer with citations like (page X).
If not found, write exactly: Not found in the provided PDF.
"""
    draft = draft_llm.invoke(prompt).content.strip()
    return {"evidence": evidence, "draft": draft}


In [6]:
critic_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def critique(question: str, draft: str) -> str:
    prompt = f"""
You are a strict reviewer for a PDF-grounded answer.

Question:
{question}

Draft:
{draft}

Check:
1) Does every claim have a citation (page X)?
2) Any hallucinations beyond the PDF?
3) Is the answer incomplete or vague?
4) Are citations plausible and consistent?

Return:
- A bullet list of issues
- A bullet list of concrete fixes
- A risk score from 1 (safe) to 5 (high risk)
"""
    return critic_llm.invoke(prompt).content.strip()


In [7]:
revise_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def revise(question: str, draft: str, critique_text: str) -> str:
    # Second retrieval guided by critique (helps a lot)
    evidence2 = call_claim_tool(f"{question}\n\nReviewer concerns:\n{critique_text}")

    prompt = f"""
{SYSTEM_PROMPT}

Tool name: {claim_tool.name}

Question:
{question}

Draft:
{draft}

Critique:
{critique_text}

More evidence:
{evidence2}

Write the FINAL improved answer with citations like (page X).
If not found, write exactly: Not found in the provided PDF.
"""
    return revise_llm.invoke(prompt).content.strip()


In [8]:
q = "Does the transformer allow multimodality of data?"

d = draft_answer(q)
c = critique(q, d["draft"])
final = revise(q, d["draft"], c)

print("=== DRAFT ===\n", d["draft"])
print("\n=== CRITIQUE ===\n", c)
print("\n=== FINAL ===\n", final)


/Users/satyammittal/github/AI-ML/demo_applications/agent-patterns-pack/claim_verifier.py:91: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  hits = retriever.get_relevant_documents(query)


=== DRAFT ===
 Not found in the provided PDF.

=== CRITIQUE ===
 ### Issues
- The draft states "Not found in the provided PDF," which does not address the question directly.
- There are no citations provided, which is necessary to support any claims made.
- The answer is incomplete as it does not provide any information about the transformer and its capabilities regarding multimodality.
- The draft does not engage with the content of the PDF, leading to a lack of context or relevance.

### Concrete Fixes
- Review the PDF thoroughly to find relevant information about transformers and their ability to handle multimodal data.
- Provide specific citations from the PDF (e.g., "According to page X, transformers can process multimodal data...").
- Expand the answer to include a brief explanation of how transformers work with different types of data (e.g., text, images, etc.).
- Ensure that the answer is clear and directly addresses the question posed.

### Risk Score
4 (high risk) - The answe

In [9]:
q = "Does the transformer use softmax?"

d = draft_answer(q)
c = critique(q, d["draft"])
final = revise(q, d["draft"], c)

print("=== DRAFT ===\n", d["draft"])
print("\n=== CRITIQUE ===\n", c)
print("\n=== FINAL ===\n", final)


=== DRAFT ===
 Yes, the transformer uses the softmax function. It is mentioned that a learned linear transformation and softmax function are used to convert the decoder output to predicted next-token probabilities (page 4).

=== CRITIQUE ===
 ### Issues
- The claim about the transformer using the softmax function is made, but the citation is vague ("page 4") without specifying the document title or context.
- The answer does not clarify which part of the transformer architecture utilizes the softmax function, potentially leading to confusion.
- There is no mention of the context in which the softmax function is applied (e.g., during the attention mechanism or output layer).
- The phrase "mentioned that a learned linear transformation and softmax function" is somewhat vague and could be more precise.

### Concrete Fixes
- Specify the document title or context when citing "page 4" to provide clarity.
- Elaborate on where in the transformer architecture the softmax function is applied (e.